## German-Sentiment (BERT)

In [ ]:
!pip install germansentiment

In [ ]:
from germansentiment import SentimentModel
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle
model = SentimentModel()
sentiment=[]
de_df_factor = pd.read_pickle('data/de_df_factor.pkl')
de_df_factor=de_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)
pos=[]
neu=[]
neg=[]
for l in range(len(de_df_factor)):
    pos.append(0)
    neu.append(0)
    neg.append(0)
    sentiment.append(0)
for i in tqdm(range(len(de_df_factor)), position=0, leave=True):

    sentiment[i], probabilities= model.predict_sentiment([de_df_factor.text[i]], output_probabilities = True)


    for p in probabilities[0]:
        if p[0]== "neutral":
            neu[i]=p[1]
        elif p[0]=="positive":
            pos[i]=p[1]
        elif p[0]=="negative":
            neg[i]=p[1]
                
        
    

de_df_factor["sentiment"]=sentiment
de_df_factor["p_pos"]=pos
de_df_factor["p_neu"]=neu
de_df_factor["p_neg"]=neg

de_df_factor.to_pickle('data/de_df_sentiments_bert_whole_len_no_dup.pkl')
de_df_factor.head()

In [ ]:
de_df_bert=pd.read_pickle('data/de_df_sentiments_bert_whole_len_no_dup.pkl')


In [ ]:
de_df_bert.groupby('sentiment').count()['id']

## TextBlob

In [ ]:
from textblob_de import TextBlobDE as TextBlob
de_df_factor=pd.read_pickle('data/de_df_sentiments_no_dup.pkl')

de_df_factor=de_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)

class_dict=defaultdict(int)
subjectivity_dict=defaultdict(int)
sentiment=[]
subjectivity=[]
sent_stats=[]
subj_stats=[]
for i in tqdm(range(len(de_df_factor)), position=0, leave=True):
    
    classes = TextBlob(de_df_factor.text[i])
    sent=classes.sentiment.polarity
    sent_stats.append(sent)
  
    if sent<=-0.35:
        class_dict["negative"]+=1
        sentiment.append("negative")

    elif sent>=0.4:
        class_dict["positive"]+=1
        sentiment.append("positive")
      

    else:
        class_dict["neutral"]+=1 
        sentiment.append("neutral")

    sub=classes.sentiment.subjectivity
    subj_stats.append(sub)


    if sub>=0.01:
        subjectivity_dict["subjective"]+=1
        subjectivity.append("subjective")
    
    else:
        subjectivity_dict["not_subjective"]+=1 
        subjectivity.append("not_subjective")

      
    
de_df_factor["sentiment"]=sentiment
de_df_factor["sentiment_score"]=sent_stats
de_df_factor["subjectivity"]=subjectivity
de_df_factor["subjectivity_score"]=subj_stats
de_df_factor.to_pickle('data/de_df_sentiments_no_dup_3classes.pkl')
print(class_dict.items())
print(subjectivity_dict.items())


In [ ]:
len(sent_stats)
print(min(sent_stats))
print(max(sent_stats))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=sent_stats)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=sent_stats,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([-0.35,0.35])

In [ ]:
len(subj_stats)
print(min(subj_stats))
print(max(subj_stats))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=subj_stats)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=subj_stats,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([0,0.05])

In [ ]:
from xml.etree import cElementTree
def load(path):
        """ Loads the XML-file (with sentiment annotations) from the given path.
            By default, Sentiment.path is lazily loaded.
        """
        # <word form="great" wordnet_id="a-01123879" pos="JJ" polarity="1.0" subjectivity="1.0" intensity="1.0" />
        # <word form="damnmit" polarity="-0.75" subjectivity="1.0" label="profanity" />
        
        if not os.path.exists(path):
            return
        words, synsets, labels = {}, {}, {}
        xml = cElementTree.parse(path)
        xml = xml.getroot()
        lex={}
        for w in xml.findall("word"):
            
                w, s  = (
                    w.attrib.get("form"),
                    w.attrib.get("subjectivity", 0.0),
                    
                )
                if float(s)>0.0:
                    lex[w]=s
        
        return lex     
lookup=load("data/de-sentiment.xml")
print(lookup.keys())

In [ ]:
#!pip install wordcloud
subj_df=textblob_df[(textblob_df["subjectivity_score"]>=0.1)]
subj_df.reset_index(drop=True, inplace=True)
print(len(subj_df))
from nltk import word_tokenize, sent_tokenize
subjsent=[]
for i in range(len(subj_df)):
    sentences=sent_tokenize(subj_df.loc[i].text, language="german")
    
    for s in sentences:
        token=word_tokenize(s, language="german")
        
        for tok in token:
            if tok.lower() in lookup.keys():
                  subjsent.append(s)

subs=" ".join(list(subjsent))
text = subs
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import os

d = os.path.dirname(__file__) if "__file__" in locals() else os.getcwd()



nichtinteressant = "e es war H vor zum keiner sowie aufgrund Oktober Juni dem hatte zurück dass April September stellte diese einem eines wegen seine diese November sie einer über B da einen auch sich als er ein ihm seien X März Dezember Februar IV zur welche weshalb wie bei habe weil ihr aus dieser nach dem Januar Mai Juli sei Nachdem August und von Der eine vom ab nicht mit mehr Fr des zu für seit viel keine bis das den wir ist die auf im"
liste_der_unerwuenschten_woerter = nichtinteressant.split()

STOPWORDS.update(liste_der_unerwuenschten_woerter)
wordcloud = WordCloud(background_color="white",width=1920, height=1080).generate(text)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
print(subjsent)
print(len(subjsent))

In [ ]:
import pandas as pd
textblob_df=pd.read_pickle('data/de_df_sentiments_no_dup_3classes.pkl')
from collections import defaultdict
textblob_dict=defaultdict(int)
t_list=[]
for t in textblob_df["sentiment"]:
    textblob_dict[t]+=1
textblob_dict



In [ ]:
import pandas as pd
textblob_df=pd.read_pickle('data/de_df_sentiments_no_dup_3classes.pkl')
from collections import defaultdict
textblob_dict=defaultdict(int)
t_list=[]
for t in textblob_df["subjectivity"]:
    textblob_dict[t]+=1
textblob_dict

## GerVADER

In [ ]:
from GerVADER.vaderSentimentGER import SentimentIntensityAnalyzer

#standard values in vader are:
    #positive sentiment: compound score >= 0.05
    #neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    #negative sentiment: compound score <= -0.05


analyzer = SentimentIntensityAnalyzer()
import spacy
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle
de_nlp = spacy.load('de_core_news_lg')
config = {"punct_chars": None}
de_nlp.add_pipe("sentencizer", config=config)
sentiment=[]
#only 3 classes to compare with other classifiers later
gervader=[]
n=[]
p=[]
neu=[]
vader_scores=[]
de_df_factor = pd.read_pickle('data/de_df_factor.pkl')
de_df_factor=de_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)
from statistics import mean
from collections import defaultdict
for i in tqdm(range(len(de_df_factor)), position=0, leave=True):
    senti=defaultdict(int)
    doc=de_nlp(de_df_factor.text[i])
    senti["positive"]=0
    senti["negative"]=0
    senti["neutral"]=0
    for sentence in doc.sents:
        sents=[]
        vs=analyzer.polarity_scores(sentence.text)
        
        sent=vs["compound"]
        vader_scores.append(sent)
        #we decided to not be so granular and only take negative, neutral and positive
        """
        if sent>=0.5:
            senti["strong_positive"]+=1
        elif sent<=-0.5:
            senti["strong_negative"]+=1
        elif sent>-0.5 and sent<=-0.3:
            senti["negative"]+=1

        elif sent<0.5 and sent>=0.3:     
            senti["positive"]+=1

        elif sent>-0.3 and sent<=-0.1:
            senti["slightly_negative"]+=1

        elif sent<0.3 and sent>=0.1:
            senti["slightly_positive"]+=1
        """
        if sent>=0.4:
            senti["positive"]+=1
        elif sent<=-0.4:
            senti["negative"]+=1
        else:
            senti["neutral"]+=1

    final_class=max(senti, key=senti.get)

    p.append(senti["positive"]/len(list(doc.sents)))
    n.append(senti["negative"]/len(list(doc.sents)))
    neu.append(senti["neutral"]/len(list(doc.sents)))
    gervader.append(final_class)

de_df_factor["gervader_sentiment"]=gervader
de_df_factor["positive"]=p
de_df_factor["negative"]=n
de_df_factor["neutral"]=neu

de_df_factor.to_pickle('data/de_df_sentiments_gervader_no_dup_3classes04.pkl')
de_df_factor.head()



In [ ]:
gervader_df=pd.read_pickle('data/de_df_sentiments_gervader_no_dup_3classes04.pkl')

In [ ]:
len(vader_scores)
print(min(vader_scores))
print(max(vader_scores))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=vader_scores)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=vader_scores,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([-0.3,0.3])

## SentiWS

In [ ]:
import spacy
from spacy_sentiws import spaCySentiWS
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import pickle
de_nlp = spacy.load('de_core_news_lg')
de_nlp.add_pipe('sentiws', config={'sentiws_path': 'data/sentiws/'})
config = {"punct_chars": None}
de_nlp.add_pipe("sentencizer", config=config)
sentiment=[]

n=[]
p=[]

neu=[]
sent_stats=[]
de_df_factor = pd.read_pickle('data/de_df_factor.pkl')
de_df_factor=de_df_factor.drop_duplicates(subset=['year',"text","label","region","canton","category","language"], ignore_index=True)
from statistics import mean
from collections import defaultdict
for i in tqdm(range(len(de_df_factor)), position=0, leave=True):
    senti=defaultdict(int)
    doc=de_nlp(de_df_factor.text[i])
    
    senti["positive"]=0
    senti["negative"]=0
    senti["neutral"]=0
    for sentence in doc.sents:
        sents=[]
        
        for token in sentence:
            if token._.sentiws:
                sents.append(token._.sentiws)
            else:
                sents.append(0)
        sent=mean(sents)
        sent_stats.append(sent)
     
        if sent<-0.01:
            senti["negative"]+=1

        elif sent>0.01:     
            senti["positive"]+=1

        else:
            senti["neutral"]+=1
    sentiment.append(max(senti, key=senti.get))

    p.append(senti["positive"]/len(list(doc.sents)))
    n.append(senti["negative"]/len(list(doc.sents)))
    neu.append(senti["neutral"]/len(list(doc.sents)))

   
de_df_factor["sentiws_sentiment"]=sentiment

de_df_factor["positive"]=p
de_df_factor["negative"]=n

de_df_factor["neutral"]=neu




de_df_factor.to_pickle('data/de_df_sentiments_sentiws_no_dup_new.pkl')
de_df_factor.head()


In [ ]:
len(sent_stats)
print(min(sent_stats))
print(max(sent_stats))
import seaborn as sns
import matplotlib.pyplot as plt
sns.violinplot(x=sent_stats)
ax2 = plt.axes([0.2, 0.6, .2, .2], facecolor='y')
sns.violinplot(x=sent_stats,ax=ax2)
ax2.set_title('zoom')
ax2.set_xlim([-0.025,0.025])




In [ ]:
sentiws_df = pd.read_pickle('data/de_df_sentiments_sentiws_no_dup_new.pkl')

## Inter-Annotator Agreement

In [ ]:
from sklearn.metrics import cohen_kappa_score
print("Pairwise Cohen Kappa:")
print(str("SentiWS and GerVADER: "+str(cohen_kappa_score(sentiws_df["sentiws_sentiment"],gervader_df["gervader_sentiment"]))))
print(str("SentiWS and TextBlob: "+str(cohen_kappa_score(sentiws_df["sentiws_sentiment"],textblob_df["textblob_sentiment"]))))
print(str("SentiWS and BERT: "+str(cohen_kappa_score(sentiws_df["sentiws_sentiment"],de_df_bert["sentiment"]))))
print(str("GerVADER and TextBlob: "+str(cohen_kappa_score(gervader_df["gervader_sentiment"],textblob_df["textblob_sentiment"]))))
print(str("GerVADER and BERT: "+str(cohen_kappa_score(gervader_df["gervader_sentiment"],de_df_bert["sentiment"]))))
print(str("TextBlob and BERT: "+str(cohen_kappa_score(textblob_df["textblob_sentiment"],de_df_bert["sentiment"]))))




In [ ]:
#fleiss kappa
from nltk import agreement

s1=sentiws_df["sentiws_sentiment"].tolist()
s2=gervader_df["gervader_sentiment"].tolist()
s3=textblob_df["textblob_sentiment"].tolist()
s4=de_df_bert["sentiment"].tolist()

formatted_codes = [[1,i,s1[i]] for i in range(len(s1))] + [[2,i,s2[i]] for i in range(len(s2))]  + [[3,i,s3[i]] for i in range(len(s3))]+ [[4,i,s4[i]] for i in range(len(s4))]
ratingtask = agreement.AnnotationTask(data=formatted_codes)
print('Fleiss\'s Kappa:',ratingtask.multi_kappa())